<a href="https://colab.research.google.com/github/man1sh1409/fakeNewsSocialMediaProspective/blob/main/fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [2]:
# read all the datasets
df1=pd.read_csv('/content/gossipcop_real.csv')
df2=pd.read_csv('/content/politifact_real.csv')
df3=pd.read_csv('/content/politifact_fake.csv')
df4=df1=pd.read_csv('/content/gossipcop_fake.csv')

In [54]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5323 non-null   object
 1   news_url   5067 non-null   object
 2   title      5323 non-null   object
 3   tweet_ids  5135 non-null   object
dtypes: object(4)
memory usage: 166.5+ KB


In [55]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [53]:
#combine the real and fake datasets
df_real=pd.concat([df1,df2],axis=0)
df_fake=pd.concat([df3,df4],axis=0)

In [56]:
# check for null and empty value
df_real.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5947 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5947 non-null   object
 1   news_url   5634 non-null   object
 2   title      5947 non-null   object
 3   tweet_ids  5544 non-null   object
dtypes: object(4)
memory usage: 232.3+ KB


In [57]:
# check data sample of real
df_real.sample(5)

,id,news_url,title,tweet_ids
2464,gossipcop-6780720672,www.hellomagazine.com/healthandbeauty/mother-a...,Beyonce looks amazing in Instagram snap: defia...,NaN
969,gossipcop-4606615612,www.instyle.com/news/justin-theroux-rumored-da...,Justin Theroux Isn't Dating 9 Different Women—...,972383454720614400\t972896416820486144\t972896...
4227,gossipcop-8159726226,www.imdb.com/news/ni61555569,"Justin Timberlake, Jessica Biel Not Getting “$...",913216854432284672\t913331833735045120\t913332...
2110,gossipcop-2088192726,people.com/celebrity/nina-agdal-christie-brink...,Are Nina Agdal and Jack Brinkley Cook Dating?,NaN
3700,gossipcop-8021032353,www.eonline.com/news/901194/vanessa-hudgens-ad...,Vanessa Hudgens Addresses Speculation She's En...,943214359798603781\t943224191758745600\t943226...


In [62]:
df_real.shape

(5501, 4)

In [59]:
# remove title having less than 5 words
df_real=df_real[~df_real.title.str.count('\s+').lt(4)]

In [61]:
# check shape of real datasets
df_real.shape

(5501, 4)

In [63]:
# add label column for real news
df_real['label']=0
df_real.head()

,id,news_url,title,tweet_ids,label
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,0
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,0
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,0
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,0


In [65]:
# similarly for fake news also
df_fake=df_fake[~ df_fake.title.str.count('\s+').lt(4)]
# add label column
df_fake['label']=1
df_fake.head()

,id,news_url,title,tweet_ids,label
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,1
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,1
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,1
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,1
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,1


In [66]:
# make a dataset as a whole so that we can train our model
df=pd.concat([df_real,df_fake])

In [67]:
# shuffle the final dataset to ovecome the over fitting 
df=df.sample(frac=1)
df.head()

,id,news_url,title,tweet_ids,label
2230,gossipcop-4084108386,hollywoodlife.com/2018/02/01/selena-gomez-piss...,Selena Gomez Pissed Charlie Puth Went Public W...,959216716571168768\t959216778072281088\t959217...,0
2058,gossipcop-7499447250,www.tmz.com/person/tarek-el-moussa/,"Tarek El Moussa News, Pictures, and Videos",991845589246619648\t991945477313712128,1
1687,gossipcop-612921286,www.justjared.com/2017/11/27/nominees-robert-p...,Nominees Robert Pattinson & James Franco Are S...,935326683279904768\t935327907215441920\t935328...,0
3795,gossipcop-393798271,www.yahoo.com/entertainment/heres-a-complete-h...,Here’s a Complete History of Jennifer Lopez an...,705932619670691841\t705932622547955712\t705934...,0
362,gossipcop-4568998185,www.etonline.com/blake-shelton-embarrasses-gwe...,Blake Shelton Embarrasses Gwen Stefani Onstage...,914641442860961792\t914642885617664002\t914644...,1


In [68]:
# count the distinct label value
df.label.value_counts()

0    5501
1    5433
Name: label, dtype: int64

In [70]:
# download stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [71]:
# download wordNet for english language from laxical database
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [74]:
#Text preprocessing
def preprocessing(tweet):
  text=BeautifulSoup(tweet).get_text() # Remove html tags
  text=re.sub("[^a-zA-Z#]"," ",text) # remove special characters
  test=re.sub('((www.[^s]+) | (https?://[^s]+))',' ',text) # remove urls
  text=text.lower().split() # convert to lowercase and split each words
  
  stop_w=set(stopwords.words("english")) # set of all stopwords for fast search
  text=[w for w in text if not w in stop_w] # reomve stop words
  text=[WordNetLemmatizer().lemmatize(w) for w in text] # Lemmatization if each word

  return (" ".join(text)) # return words after joining with each word separator



In [75]:
# clean the title of dataset
df['title']=df['title'].apply(lambda text: preprocessing(text))
df['title'].head()

<ipython-input-74-b7e35b1add9c>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text=BeautifulSoup(tweet).get_text() # Remove html tags


2230    selena gomez pissed charlie puth went public r...
2058                   tarek el moussa news picture video
1687    nominee robert pattinson james franco scruffy ...
3795    complete history jennifer lopez mariah carey a...
362     blake shelton embarrasses gwen stefani onstage...
Name: title, dtype: object

In [78]:
# spliting the data into train and test 

train,test=train_test_split(df,test_size=0.2,stratify=df['label'],random_state=42)
train.shape,test.shape

((8747, 5), (2187, 5))

In [80]:
# Create a TF-IDF vectorizer object
tfidf_vec = TfidfVectorizer(max_features=10000, ngram_range=(1, 3))

# Data fitting and transformation
train_df = tfidf_vec.fit_transform(train.title)
test_df  = tfidf_vec.transform(test.title)

In [81]:
# before over-sampling the minority class
train['label'].value_counts()

0    4401
1    4346
Name: label, dtype: int64

In [97]:
x_train,y_train=train.iloc[:,:-1],train.iloc[:,-1:]
x_train=np.array(x_train['title'])
y_train=np.array(y_train)
x_test,y_test=test.iloc[:,:-1],test.iloc[:,-1:]
x_test=np.array(x_test['title'])
y_test=np.array(y_test)
x_train

array(['brad pitt doubtful angelina jolie successful th marriage failed',
       'robert pattinson desperately want bond kristen stewart',
       'brad pitt angelina jolie reach custody settlement month bitter divorce row',
       ...,
       'taylor swift gave sneaky response internet joe alwyn breakup speculation',
       'exactly katie holmes relationship tabloid',
       'voice thing know gwen stefani blake shelton relationship'],
      dtype=object)

In [100]:
# Use SMOTE (Synthetic Minority Oversampling Technique) for dealing with class imbalance
smt = SMOTE(random_state = 18, sampling_strategy = 1.0)
smt_xtrain_df, smt_ytrain = smt.fit_resample(train_df, train.label)
smt_ytrain.value_counts()

1    4401
0    4401
Name: label, dtype: int64

In [101]:
# fit into model
# MOdel 1: Logistic Regressor
logis_reg=LogisticRegression()
logis_reg.fit(smt_xtrain_df,smt_ytrain)

LogisticRegression()

In [102]:
y_pred=logis_reg.predict(test_df)

In [103]:
# evaluation metrics
print(classification_report(test.label, y_pred, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.22      0.21      0.22      1100
    Positive       0.24      0.25      0.24      1087

    accuracy                           0.23      2187
   macro avg       0.23      0.23      0.23      2187
weighted avg       0.23      0.23      0.23      2187



In [104]:
# Model 2 - Linear Support Vector Classifier
linear_svc = LinearSVC()
linear_svc.fit(smt_xtrain_df, smt_ytrain)

y_pred = linear_svc.predict(test_df)

# Evaluation metrics
print(classification_report(test.label, y_pred, target_names=['Negative', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.18      0.18      0.18      1100
    Positive       0.18      0.18      0.18      1087

    accuracy                           0.18      2187
   macro avg       0.18      0.18      0.18      2187
weighted avg       0.18      0.18      0.18      2187

